In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.quick_scripts.GetNRL import getNRL
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity, parseNucleosomeCountsDataForPlotting, plotPeriodicity

adarDamageSeqDataDirectory = os.path.join(getDataDir(), "Adar_damage-seq")

dataSets = ["A549_BPDE_2h"]

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

adarDamageSeqRawCounts = dict()
adarDamageSeqNormalizedCounts = dict()
NRLs = dict()

nucleosomeMapNameWhitelist = ['hg19_hybrid_nuc_map', 'hg19_LCL_MNase_nuc_map', 'hg19_NHF1_MNase_nuc_map']
# nucleosomeMapNameWhitelist += ['hg19_LCL_MNase_nuc_map_euchromatin', 'hg19_LCL_MNase_nuc_map_heterochromatin',
#                               'hg19_LCL_MNase_nuc_map_stringent_euchromatin']

for dataSet in dataSets:

    adarDamageSeqRawCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
    adarDamageSeqNormalizedCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}

    for nucleosomeCountsFilePath in getFilesInDirectory(adarDamageSeqDataDirectory, DataTypeStr.rawNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in nucleosomeMapNameWhitelist: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        adarDamageSeqRawCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

        if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

    for nucleosomeCountsFilePath in getFilesInDirectory(adarDamageSeqDataDirectory, DataTypeStr.normNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in nucleosomeMapNameWhitelist: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        adarDamageSeqNormalizedCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

        if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

In [ ]:
for dataSet in dataSets:
    for radius in adarDamageSeqRawCounts[dataSet]:
        for nucleosomeMapName in adarDamageSeqRawCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(adarDamageSeqRawCounts[dataSet][radius][nucleosomeMapName], dataCol = "Aligned_Strands_Counts",
                                           smoothTranslational = False, nucRepLen = NRLs[nucleosomeMapName],
                                           title = f"Adar {dataSet} {nucleosomeMapName} Raw Counts", yAxisLabel = "Damage Reads")
            print(plot)

In [ ]:
for dataSet in dataSets:
    for radius in adarDamageSeqNormalizedCounts[dataSet]:
        for nucleosomeMapName in adarDamageSeqNormalizedCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(adarDamageSeqNormalizedCounts[dataSet][radius][nucleosomeMapName], dataCol = "Normalized_Both_Strands",
                                           smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                           title = f"Adar {dataSet} {nucleosomeMapName} Naked Normalized", yAxisLabel = "Cellular/Naked Damage Reads")
            print(plot)